This is a brief demonstration of testing in Motmot. We'll consider both hand-written test cases (using the `Testing` library) and property-based cases (using the `QuickCheck` library).

In [1]:
:setup-notebook

│                                Welcome to Motmot
│              Copyright 2015–2023, K.D.P.Ross <KDPRoss@gmail.com>
│                       Designed in Paris; built in Tucson.
│
│                            T6=7bf2e32 | Libs=a3b9a08
│
│                         :help / :h — Print REPL help.
│                        :file / :f — Load a script file.
│                       :quit / :q — Exit the interpreter.
│                     :reset / :r — Reset interpreter state.
│
│ Libraries loaded.
│   All.All, Chars.Diacritics, Chars.Extended, Chars.Inverse, Compute.Tree,
│   Compute.Units, Data.Table, Document.Rendering, Document.Structure,
│   FileProcessing, Latex.Colour, Latex.Process, Monads, Parser, Prelude.doc,
│   PreludeExtended.doc, Processing.DSL, Processing.Text, Processing.Types,
│   Rendering.Types, Syntax.Meta, Syntax.Punctuation, Syntax.Standard

In [2]:
-- We'll need something to test. Let's write a very simple
-- greedy-algorithm word wrapper for this.
wrap : Num -> String -> [ String ] =
  width : Num ~
    words >> core [ [{ String }] ]
  where core : [ [ String ] ] -> [ String ] -> [ String ] =
          ((res : [ [ String ] ]) and (cur : [ String ]) :: (rest : [ [ String ] ])) ~
            fun
            -- Base case: Nothing left to process; reverse the lines and
            -- smoosh them together.
            ([{ String }] ~ res |>
                              filter (
                                empty?{ String } >>
                                not
                              ) >> reverse >>
                              map (
                                reverse{ String } >>
                                concat-words
                              )
            )
            -- Fun edge case: If we've a single word longer than the
            -- length, put it alone on a line.
            (((x : String) and
              {length-string >> (_ >= width)}
             ) :: (xs : [ String ])           ~ core ([ x ] :: res') xs
                                                where res' : [ [ String ] ] = drop-while empty?{ String } res
            )
            -- Know: Still words to be processed; if line would be too
            -- long, create another one.
            ((xs : [ String ]) and 
             (x : String) :: (_ : [ String ]) and
             `{length-line (x :: cur) > width}    ~ core ([{ String }] :: res) xs
            )
            -- Ordinary case; keep chugging.
            ((x : String) :: (xs : [ String ]) ~ core ((x :: cur) :: rest) xs)
      | length-line : [ String ] -> Num =
          xs : [ String ] ~
            sum (map length-string xs) + (max 0 (length xs - 1))

parsed: `(wrap : Num -> String -> [ String ]) = width : Num ~ words >> core [ [{ String }] ] where core : [ [ String ] ] -> [ String ] -> [ String ] = ((res : [ [ String ] ]) and (cur : [ String ]) :: (rest : [ [ String ] ])) ~ fun ([{ String }] ~ res |> filter (empty?{ String } >> not) >> reverse >> map (reverse{ String } >> concat-words)) (((x : String) and {length-string >> (_ >= width)}) :: (xs : [ String ]) ~ core ([ x ] :: res') xs where res' : [ [ String ] ] = drop-while empty?{ String } res) ((xs : [ String ]) and (x : String) :: (_ : [ String ]) and `{length-line (x :: cur) > width} ~ core ([{ String }] :: res) xs) ((x : String) :: (xs : [ String ]) ~ core ((x :: cur) :: rest) xs) | length-line : [ String ] -> Num = xs : [ String ] ~ sum (map length-string xs) + (max 0 (length xs - 1))`
has type: `Num -> String -> [ String ]`
linearised: <fun>

In [3]:
string-input some-text
wrap : Num -> String -> [ String ] =
  width : Num ~
    words >> core [ [{ String }] ]
  where core : [ [ String ] ] -> [ String ] -> [ String ] =
          ((res : [ [ String ] ]) and (cur : [ String ]) :: (rest : [ [ String ] ])) ~
            fun
            ([{ String }] ~ res |>
                              filter (
                                empty?{ String } >>
                                not
                              ) >> reverse >>
                              map (
                                reverse{ String } >>
                                concat-words
                              )
            )
            (((x : String) and
              {length-string >> (_ >= width)}
             ) :: (xs : [ String ])           ~ core ([ x ] :: res') (trace xs)
                                                where res' : [ [ String ] ] = drop-while empty?{ String } res
            )
            ((xs : [ String ]) and 
             (x : String) :: (_ : [ String ]) and
             `{length-line (x :: cur) > width}    ~ core ([{ String }] :: res) xs
            )
            ((x : String) :: (xs : [ String ]) ~ core ((x :: cur) :: rest) xs)
      | length-line : [ String ] -> Num =
          xs : [ String ] ~
            sum (map length-string xs) + (max 0 (length xs - 1))

In [4]:
-- What could be more fun than using some code to wrap
-- itself?
some-text |>
  concat-words >>
  wrap 60 >>
  iter print{ String }

parsed: `some-text |> concat-words >> wrap 60 >> iter print{ String }`
has type: `()`
PRINT wrap : Num -> String -> [ String ] = width : Num ~ words >>
PRINT core [ [{ String }] ] where core : [ [ String ] ] -> [
PRINT String ] -> [ String ] = ((res : [ [ String ] ]) and (cur :
PRINT [ String ]) :: (rest : [ [ String ] ])) ~ fun ([{ String }]
PRINT ~ res |> filter ( empty?{ String } >> not ) >> reverse >>
PRINT map ( reverse{ String } >> concat-words ) ) (((x : String)
PRINT and {length-string >> (_ >= width)} ) :: (xs : [ String ]) ~
PRINT core ([ x ] :: res') (trace xs) where res' : [ [ String ] ]
PRINT = drop-while empty?{ String } res ) ((xs : [ String ]) and
PRINT (x : String) :: (_ : [ String ]) and `{length-line (x ::
PRINT cur) > width} ~ core ([{ String }] :: res) xs ) ((x :
PRINT String) :: (xs : [ String ]) ~ core ((x :: cur) :: rest) xs)
PRINT | length-line : [ String ] -> Num = xs : [ String ] ~ sum
PRINT (map length-string xs) + (max 0 (length xs - 1))
linearised: ()

In [5]:
-- Great, now, we can get to testing. Let's first import the
-- libraries that we'll need.
:file Testing.Extended ;
:file QuickCheck.Extended ;
:open QuickCheck ;

In [6]:
-- Let's write a few cases by hand:

run-test-eq $`wrap` tests$ (
  uncurry wrap
) [
  -- Some 'happy-path' test cases.
  (10, $sphinx of black quartz judge my vow$)   |-> [ $sphinx of$, $black$, $quartz$, $judge my$, $vow$ ],
  (20, $sphinx of black quartz judge my vow$)   |-> [ $sphinx of black$, $quartz judge my vow$ ],
  (80, $sphinx of black quartz judge my vow$)   |-> [ $sphinx of black quartz judge my vow$ ],
  -- Should normalise spaces.
  (80, $ sphinx of black quartz judge my vow $) |-> [ $sphinx of black quartz judge my vow$ ],
  (80, $sphinx of black   quartz judge my vow$) |-> [ $sphinx of black quartz judge my vow$ ],
  -- Edge cases.
  (10, $$)                                      |-> [{ String }],
  (0, $hi$)                                     |-> [ $hi$ ],
  (-20, $hi$)                                   |-> [ $hi$ ],
  (10.5, $sphinx of black quartz judge my vow$) |-> [ $sphinx of$, $black$, $quartz$, $judge my$, $vow$ ],
]
using Testing

parsed: `run-test-eq $`wrap` tests$ (uncurry wrap) [ (10, $sphinx of black quartz judge my vow$) |-> [ $sphinx of$, $black$, $quartz$, $judge my$, $vow$ ], (20, $sphinx of black quartz judge my vow$) |-> [ $sphinx of black$, $quartz judge my vow$ ], (80, $sphinx of black quartz judge my vow$) |-> [ $sphinx of black quartz judge my vow$ ], (80, $ sphinx of black quartz judge my vow $) |-> [ $sphinx of black quartz judge my vow$ ], (80, $sphinx of black   quartz judge my vow$) |-> [ $sphinx of black quartz judge my vow$ ], (10, $$) |-> [{ String }], (0, $hi$) |-> [ $hi$ ], (-20, $hi$) |-> [ $hi$ ], ((10 + 1 / 2), $sphinx of black quartz judge my vow$) |-> [ $sphinx of$, $black$, $quartz$, $judge my$, $vow$ ] ] using Testing`
has type: `()`
PRINT [`wrap` tests]: Ran 9 cases successfully.
linearised: ()

In [7]:
-- Let's write some QuickCheck cases; they're
-- 'self-documenting by design'.

assert pred using(QuickCheck.Arb)(pair (natural >>> (_ mod pragmatic-limit)) (list string)) (
  $output has correct words in correct order$,
  for all (n : Num) (xs : [ String ]) ~
    xs == (xs |> concat-words >> wrap n >> concat-map words)
)
where pragmatic-limit : Num = 100

assert pred using(QuickCheck.Arb)(pair (natural >>> (_ mod pragmatic-limit)) (list string >>> concat-words)) (
  $output lines are no longer than width or single too-long word$,
  for all (n : Num) (s : String) ~
    s |>
      wrap n >>
      forall (s : String ~
        length-string s =< n or not (mr $ $ s)
      )
)
where pragmatic-limit : Num = 100

assert pred using(QuickCheck.Arb)(pair rational (list string >>> concat-words)) (
  $wrapping with a rational width equiv to truncation$,
  for all (q : Num) (s : String) ~
    wrap (floor q) s == wrap q s
)

parsed: `assert_pred using(QuickCheck.Arb)(pair (natural >>> (_ mod pragmatic-limit)) (list string)) ($output has correct words in correct order$, for all (n : Num) (xs : [ String ]) ~ xs == (xs |> concat-words >> wrap n >> concat-map words)) where pragmatic-limit : Num = 100`
has type: `()`
PRINT Case 'output has correct words in correct order'
PRINT Okay, passed 100 tests.
linearised: ()

parsed: `assert_pred using(QuickCheck.Arb)(pair (natural >>> (_ mod pragmatic-limit)) (list string >>> concat-words)) ($output lines are no longer than width or single too-long word$, for all (n : Num) (s : String) ~ s |> wrap n >> forall (s : String ~ length-string s =< n or not (mr $ $ s))) where pragmatic-limit : Num = 100`
has type: `()`
PRINT Case 'output lines are no longer than width or single too-long word'
PRINT Okay, passed 100 tests.
linearised: ()

parsed: `assert_pred using(QuickCheck.Arb)(pair rational (list string >>> concat-words)) ($wrapping with a rational width equiv to truncation